In [17]:
from ngsolve import *
# %gui tk
from ngsolve.webgui import *
# from ngsolve import Draw, Redraw
from netgen.geom2d import unit_square
from ngsolve.solvers import *
from netgen.geom2d import SplineGeometry


# ngsglobals.msg_level = 1

# generate a triangular mesh of mesh-size 0.1
# mesh = Mesh(unit_square.GenerateMesh(maxh=0.4/2/2))

# Generate the background mesh
square = SplineGeometry()
square.AddRectangle((2, 2), (3, 3), bcs= [1,2,3,4])
# square.AddRectangle((-2, -2), (2, 2), bcs=[1, 2, 3, 4])
mesh = Mesh(square.GenerateMesh(maxh=0.4/2, quad_dominated=False))

# H1-conforming finite element space
fes = H1(mesh, order=1, dirichlet="default")

# coefficient function
# f = (-2*x*x)*((-1+x)**2)*(y**2)*((-1+y)**2)*(x**2 + y**2 - x - y)
# f = -2*x**2*(-1 + x)**2*y**2*(-1 + y)**2*(x**2 + y**2 - x - y)
# rhs = (-10*y**4 + 20*y**3 - 12*y**2 + 2*y)*x**6 + (30*y**4 - 60*y**3 + 36*y**2 - 6*y)*x**5 + (-10*y**6 + 30*y**5 - 60*y**4 + 70*y**3 - 36*y**2 + 6*y)*x**4 + (20*y**6 - 60*y**5 + 70*y**4 - 40*y**3 + 12*y**2 - 2*y)*x**3 + (-12*y**6 + 36*y**5 - 36*y**4 + 12*y**3 - 2)*x**2 + (2*y**6 - 6*y**5 + 6*y**4 - 2*y**3 + 2)*x - 2*y**2 + 2*y
# exact = x*(1-x)*y*(1-y)
# exact_g = CoefficientFunction(((1 - x)*y*(1 - y) - x*y*(1 - y),x*(1 - x)*(1 - y) - x*(1 - x)*y))

exactu = x**2+y**2
exact_g = CoefficientFunction((2*x,2*y))
rhs =-8*(x**2 + y**2 + 1)*x**2 - 4*(x**2 + y**2 + 1)**2 - 8*(x**2 + y**2 + 1)*y**2

# exactu = x*(1-x)*y*(1-y)
# exact_g = CoefficientFunction(((1 - x)*y*(1 - y) - x*y*(1 - y),x*(1 - x)*(1 - y) - x*(1 - x)*y))
# rhs = -((1 - x)*y*(1 - y) - x*y*(1 - y))**2 + 2*(x*(1 - x)*y*(1 - y) + 1)*y*(1 - y) - (x*(1 - x)*(1 - y) - x*(1 - x)*y)**2 + 2*(x*(1 - x)*y*(1 - y) + 1)*x*(1 - x)

# rhs = 2*y*(1 - y) + 2*x*(1 - x)

# define trial- and test-functions
u = fes.TrialFunction()
v = fes.TestFunction()
gfu = GridFunction(fes)
g = 0

# the bilinear-form 
a = BilinearForm(fes)
a += SymbolicBFI( (u+1)**2*grad(u)*grad(v) - rhs*v )
#a += (grad(u)*grad(v) - x*v)*dx
a.Assemble()

# gfu.Set(x*(1-x)*y*(1-y))
gfu.Set(exactu, BND)

Newton(a,gfu,freedofs=fes.FreeDofs(),maxit=1000,maxerr=1e-13,dampfactor=1,printing=True)
Draw(gfu,mesh,"uh")
Draw(exactu,mesh,"u")

print ("L2-error:", sqrt (Integrate ( (gfu-exactu)*(gfu-exactu), mesh)))
print ("H1-error:", sqrt (Integrate ( (grad(gfu)-exact_g)**2, mesh)))



# # Poisson
# a += grad(u)*grad(v)*dx
# a.Assemble()

# f = LinearForm(fes)
# f += rhs*v*dx
# f.Assemble()

# gfu.vec.data += a.mat.Inverse(fes.FreeDofs()) * f.vec  #,inverse="umfpack"
# Draw(gfu,mesh,"u")

# exact = x*(1-x)*y*(1-y)
# exact_g = CoefficientFunction(((1 - x)*y*(1 - y) - x*y*(1 - y),x*(1 - x)*(1 - y) - x*(1 - x)*y))
# print ("L2-error:", sqrt (Integrate ( (gfu-exact)*(gfu-exact), mesh)))
# print ("H1-error:", sqrt (Integrate ( (grad(gfu)-exact_g)**2, mesh)))


Newton iteration  0
err =  1366.7543914922153
Newton iteration  1
err =  19511.657086336483
Newton iteration  2
err =  8670.910811390902
Newton iteration  3
err =  3852.2514251241555
Newton iteration  4
err =  1707.234419616213
Newton iteration  5
err =  737.7453299297857
Newton iteration  6
err =  269.3971643043467
Newton iteration  7
err =  54.03139672483208
Newton iteration  8
err =  2.9475313394636853
Newton iteration  9
err =  0.013251107585235303
Newton iteration  10
err =  3.668872885321871e-07
Newton iteration  11
err =  9.430619104069554e-14


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

L2-error: 0.0067178009756212066
H1-error: 0.13355677939373586


In [ ]:
L2-error: 0.025839527432358
H1-error: 0.2583140158516106

L2-error: 0.0067178009756212066
H1-error: 0.13355677939373586